# LangChain for LLM Application Development from DeepLearning.AI
## Lesson 3: Chains

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

load_dotenv('OPENAI_API_KEY.env')  # Load the specified .env file
print(os.environ)  # Print all environment variables
print(os.environ.get('OPENAI_API_KEY'))  # Check specifically for your key

openai.api_key = os.environ['OPENAI_API_KEY']  # Access your API key


environ({'TMPDIR': '/var/folders/9k/956xnzkj6gj30_bckgwcyl3m0000gn/T/', '__CFBundleIdentifier': 'com.apple.Terminal', 'XPC_FLAGS': '0x0', 'LaunchInstanceID': '7DB4FF6D-56CE-42BE-86CD-6F2D0ABEAB5B', 'TERM': 'xterm-color', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.WPQ4qewqks/Listeners', 'SECURITYSESSIONID': '18e05', 'XPC_SERVICE_NAME': '0', 'TERM_PROGRAM': 'Apple_Terminal', 'TERM_PROGRAM_VERSION': '454.1', 'TERM_SESSION_ID': 'ADB6D160-EB67-4962-A917-D59C584AE1F4', 'SHELL': '/bin/zsh', 'HOME': '/Users/hclee', 'LOGNAME': 'hclee', 'USER': 'hclee', 'PATH': '/opt/anaconda3/bin:/opt/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin', 'SHLVL': '1', 'PWD': '/Users/hclee', 'OLDPWD': '/Users/hclee', 'CONDA_EXE': '/opt/anaconda3

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"
    
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd

df = pd.read_csv('Data.csv')

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

<ipython-input-7-4485047b7482>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0.9, model=llm_model)


In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

### LLM Chain
- use to retrieve end point from prompt
- can handle simultaneous translation from other languages

In [9]:
chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-9-726db833e4be>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [12]:
product ="Hervidor de Agua Eléctrico"
chain.run(product)

'"ElectroWater Boiler Co."'

In [13]:
product ="L'Or Espresso Café"
chain.run(product)

'"Espressio Creations"'

In [11]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


### SequentialChain
- run several chains one after another
- verbose: ??
- Sequential Chain handles multiple input and output

In [14]:
from langchain.chains import SimpleSequentialChain

In [29]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [30]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write 5 words description (each description max 10 words) for \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [33]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [34]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
L'Or Espresso Café Co.
1. Premium coffee experience with L'Or Espresso Café Co. 
2. Exquisite blends for coffee enthusiasts. 
3. Indulge in luxury coffee moments. 
4. Elevate your coffee experience today. 
5. Unforgettable coffee moments await you.

> Finished chain.


"1. Premium coffee experience with L'Or Espresso Café Co. \n2. Exquisite blends for coffee enthusiasts. \n3. Indulge in luxury coffee moments. \n4. Elevate your coffee experience today. \n5. Unforgettable coffee moments await you."

In [35]:
from langchain.chains import SequentialChain

In [59]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [60]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [61]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [62]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [63]:
# prompt template 5: translate follow up message to english
fifth_prompt = ChatPromptTemplate.from_template(
    "Translate the following followup_message to Mandarin:"
    "\n\n{followup_message}"
)

# chain 5: input = followup_message, language and output = english_followup
chain_five = LLMChain(llm=llm, prompt=fifth_prompt,
                     output_key="mandarin_followup"
                     )

In [64]:
# overall_chain: input= Review 
# and output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, chain_five],
    input_variables=["Review"],
    output_variables=["language", "English_Review", "followup_message", "mandarin_followup"],
    verbose=False
)

In [65]:
review = df.Review[5]
overall_chain(review)

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'language': 'French',
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'followup_message': "Cher client,\n\nNous sommes désolés d'apprendre que vous n'avez pas été satisfait de votre achat. Nous prenons très au sérieux la qualité de nos produits et nous vous assurons que nous effectuons des contrôles réguliers pour garantir leur fraîcheur et leur authenticité. Nous aimerions en savoir plus sur votre expérience afin de résoudre ce problème. Pourriez-vous s'il vous plaît nous contacter pour que nous puissions trouver une solution qui vous convienne? Votre satisfaction est notre priorité.\n\nCordialement, \nL'équipe de [Votre entreprise]",
 'mandarin_followup': '尊敬的客户，\n\n我们很抱歉得知您对购买的商品感到不满

In [66]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


### Router Chains

In [67]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [68]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [69]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [70]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [71]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [72]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [73]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [74]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [75]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

<ipython-input-75-ae9cc652db54>:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [76]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [77]:
chain.run("what are the methods from identifying prime numbers?")



> Entering new MultiPromptChain chain...
math: {'input': 'what are the methods for identifying prime numbers?'}
> Finished chain.


'There are several methods for identifying prime numbers:\n\n1. Trial division: This is the simplest method where you divide the number by all integers less than its square root and check if any of them divide evenly into the number. If none do, then the number is prime.\n\n2. Sieve of Eratosthenes: This method involves creating a list of numbers up to a certain limit and systematically crossing out multiples of each prime number, leaving only the prime numbers in the list.\n\n3. Primality tests: There are various algorithms, such as the Miller-Rabin test and the AKS primality test, that can determine if a number is prime with high probability.\n\n4. Prime factorization: By finding the prime factors of a number, you can determine if it is prime or not. If a number only has itself and 1 as factors, then it is prime.\n\nThese are just a few methods for identifying prime numbers, and each has its own advantages and limitations.'

In [78]:
chain.run("what are the different ways I can express my love to my girlfriend?")

## "None": Not within the named templates, so LLM falls back to generally trained model for answers



> Entering new MultiPromptChain chain...
None: {'input': 'what are the different ways I can express my love to my girlfriend?'}
> Finished chain.


'1. Write her a heartfelt love letter or poem expressing your feelings for her.\n2. Plan a romantic date night or surprise getaway to show her how much she means to you.\n3. Cook her favorite meal or bake her favorite dessert to show your thoughtfulness and care.\n4. Give her thoughtful gifts that show you pay attention to her likes and interests.\n5. Show physical affection through hugs, kisses, and cuddling.\n6. Compliment her and make her feel special and appreciated.\n7. Listen to her and support her in times of need.\n8. Spend quality time together doing activities she enjoys.\n9. Send her sweet text messages or leave her love notes to brighten her day.\n10. Simply tell her "I love you" and show her through your actions every day.'